In [ ]:
! pip install kaggle
! pip install autoviz
!pip install -U feature-engine

In [74]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from autoviz.AutoViz_Class import AutoViz_Class
from feature_engine.encoding import RareLabelEncoder # For encoding rare labels; we could have also used sklearn encoder with infrequent categories
from feature_engine.outliers import Winsorizer # We can use outlier handler; there is outlier_clipper also.

## 1. SK_ID_CURR Analysis:

In [76]:
# Load the dataset
#df = pd.read_csv('/content/drive/MyDrive/home-credit-default-risk/application_train.csv',header=0)
df = pd.read_csv("/content/drive/MyDrive/home-credit-default-risk/application_train.csv",header=0)
# Get the count of unique values in SK_ID_CURR
unique_count_ids = df['SK_ID_CURR'].nunique()

# Compare it with the total number of rows
total_rows = df.shape[0]

# Check if SK_ID_CURR acts as a primary key
is_primary_key = unique_count_ids == total_rows

print(f"Count of Unique values of SK_ID_CURR: {unique_count_ids}, Number of Rows: {total_rows}, Is Primary Key?: {is_primary_key}")

Count of Unique values of SK_ID_CURR: 307511, Number of Rows: 307511, Is Primary Key?: True


The unique count and total rows are the same, 307,511, so we can say that SK_ID_CURR is the main identifier.

## 2. TARGET Column Analysis:

In [77]:
# Count the occurrences of each unique value in the TARGET column
target_count = df['TARGET'].value_counts(normalize=True)

print(f"Proportions of Target Values:\n{target_count}")

Proportions of Target Values:
TARGET
0    0.919271
1    0.080729
Name: proportion, dtype: float64


The Target column is uneven because 91% of entries are 0 and only 9% are 1.

## 3. Correlation Analysis:

In [78]:
num_var=['TARGET','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','DAYS_BIRTH','DAYS_EMPLOYED','OWN_CAR_AGE','CNT_FAM_MEMBERS','APARTMENTS_AVG','COMMONAREA_AVG']
corr_matrix = df[num_var].corr()

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Pearson Correlation Matrix')
plt.show()

# Top 5 features correlated with the 'TARGET' column
target_correlation = corr_matrix['TARGET'].abs().sort_values(ascending=False)
top_correlated_features = target_correlation[1:6]
print("Top 5 features (positive and negative correlation) with the 'TARGET' column:")
print(top_correlated_features.index)

Top 5 features (positive and negative correlation) with the 'TARGET' column:
Index(['DAYS_BIRTH', 'DAYS_EMPLOYED', 'OWN_CAR_AGE', 'AMT_CREDIT',
       'APARTMENTS_AVG'],
      dtype='object')


It shows that Days_Birth, Days-Employed, Own Car Age, Amount Credit and Apartment Average have the highest correlation with the Target Variable

## 4. Histogram:

In [79]:
# Selecting five numerical features for analysis
numeric_features = ['COMMONAREA_AVG', 'DAYS_EMPLOYED', 'OWN_CAR_AGE', 'AMT_CREDIT','APARTMENTS_AVG']

# Generate histograms
plt.figure(figsize=(12, 8))
for i, feature in enumerate(numeric_features, 1):
    plt.subplot(2, 3, i)
    sns.histplot(df[feature], kde=True)
    plt.title(feature)
plt.tight_layout()
plt.show()


4 out of 5 features namely Average Common Area, Credit Amount, Apartments Average & Car age all seem to have right skewness which means that a large portion of these values are lower. Days employed seems to be left skewed which means that a large number of people have a greater width of employment

Relationship with the Target Variable

In [80]:
plt.figure(figsize=(12, 8))
for i, feature in enumerate(numeric_features, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(x='TARGET', y=feature, data=df)
    plt.title(f'Box Plot of {feature} with Target')

plt.tight_layout()
plt.show()

4 out of 5 plots seem to have a lot of outliers and the median values for Common Area Avg, Credit Amount and Apartments average are almost the same for both the Target classes.
The median Car age for Target Class 1 is greater than that of Class 0

## 5. Outlier Analysis:

In [81]:
from scipy import stats

# Selecting the variables for outlier analysis
selected_variables = ['COMMONAREA_AVG', 'DAYS_EMPLOYED', 'OWN_CAR_AGE', 'AMT_CREDIT','APARTMENTS_AVG']

# Define a function to detect outliers using z-score
def detect_outliers_zscore(data):
    outliers = []
    threshold = 3
    z_scores = stats.zscore(data)
    for i, z in enumerate(z_scores):
        if abs(z) > threshold:
            outliers.append(i)
    return outliers

# Detect outliers for each variable
outliers_dict = {}
for variable in selected_variables:
    outliers_dict[variable] = detect_outliers_zscore(df[variable])

# Print outliers for each variable
for variable, outliers in outliers_dict.items():
    print(f"Outliers in {variable}: {len(outliers)}")


Outliers in COMMONAREA_AVG: 0
Outliers in DAYS_EMPLOYED: 0
Outliers in OWN_CAR_AGE: 0
Outliers in AMT_CREDIT: 3255
Outliers in APARTMENTS_AVG: 0


In [82]:
feature=['COMMONAREA_AVG', 'DAYS_EMPLOYED', 'OWN_CAR_AGE', 'AMT_CREDIT','APARTMENTS_AVG']
plt.figure(figsize=(12, 8))
for i, feature in enumerate(numeric_features, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(y=feature, data=df)
    plt.title(f'Box Plot of {feature}')

plt.tight_layout()
plt.show()

outlier_counts = {}

# Detect outliers using IQR and count for each feature
for feature in numeric_features:
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers_count = ((df[feature] < lower_bound) | (df[feature] > upper_bound)).sum()
    outlier_counts[feature] = outliers_count
sorted_outlier_counts = sorted(outlier_counts.items(), key=lambda x: x[1], reverse=True)

print("Number of outliers for each numeric variable (descending order):")
for feature, count in sorted_outlier_counts:
    print(f"{feature}: {count}")

Number of outliers for each numeric variable (descending order):
DAYS_EMPLOYED: 72217
APARTMENTS_AVG: 10655
COMMONAREA_AVG: 7942
AMT_CREDIT: 6562
OWN_CAR_AGE: 4932


For these non normally distributed numeric variables, it looks like Employment days has the highest number of outliers

## 6. Transformation of Numeric Variables:

#### Relationship between Numeric variables and Target variable

In [83]:
# Selecting numeric features
numeric_features = ['COMMONAREA_AVG', 'DAYS_EMPLOYED', 'OWN_CAR_AGE', 'AMT_CREDIT','APARTMENTS_AVG']
# Plot the bar charts for each numeric feature against Target
plt.figure(figsize=(25, 25))
for i, feature in enumerate(numeric_features, 1):
    plt.subplot(3, 2, i)
    sns.barplot(x='TARGET', y=feature, hue = 'TARGET', data=df)
    plt.title(f'{feature} vs Target')
plt.tight_layout()
plt.show()

There is a significant difference between the two classes of the outcome for these 5 numeric variables
1. Average Common Area for Class 1 < Average Common Area for Class 0
2. Class 1 has lower number of employed days as compared to class 0
3. Car Age for type 1 is more than that of Class 0
4. Credit Amount and Average apartments are low for Class 1 as compared to class 0

#### Checking Skewness and Kurtosis


In [84]:
# Calculating skewness of numeric features before transformation
skewness_before = df[numeric_features].skew().sort_values(ascending= False)
print("Skewness before transformation:")
print(skewness_before)

Skewness before transformation:
COMMONAREA_AVG    5.457305
OWN_CAR_AGE       2.745422
APARTMENTS_AVG    2.641836
DAYS_EMPLOYED     1.664346
AMT_CREDIT        1.234778
dtype: float64


Common Area has the highest right skew among the variables and Credit amount has the lowest skew.All the variables are right skewed

In [85]:
# Calculating kurtosis of numeric features before transformation
from scipy.stats import kurtosis
for feature in numeric_features:
    kurtosis_before = kurtosis(df[feature], nan_policy='omit').round(2)
    print(f"Kurtosis for {feature}: {kurtosis_before}")

Kurtosis for COMMONAREA_AVG: 45.99
Kurtosis for DAYS_EMPLOYED: 0.77
Kurtosis for OWN_CAR_AGE: 9.21
Kurtosis for AMT_CREDIT: 1.93
Kurtosis for APARTMENTS_AVG: 11.39


The kurtosis analysis shows that Average Common Area has extremely heavy tails in the distribution, followed by Apartments_Avg and Own_Car_Age and Days Employed has the least heavy tails

#### Log transformation of specific variables

Since Average Common Area, Own Car Age & Average Apartments and Days Employed have skews >1, we shall perform log transformation for these

In [86]:
# Log transformation for specified variables
log_transform_features = ['COMMONAREA_AVG', 'OWN_CAR_AGE','APARTMENTS_AVG', 'DAYS_EMPLOYED']

# Applying log transformation to selected features
for feature in log_transform_features:
    df[feature] = np.log1p(df[feature])

# Check skewness after transformation
skewness_after = df[numeric_features].skew().round(2)
print("Skewness after transformation:")
print(skewness_after)

# Check kurtosis after transformation
for feature in numeric_features:
    kurtosis_value_after = kurtosis(df[feature], nan_policy='omit').round(2)
    print(f"Kurtosis for {feature} after transformation: {kurtosis_value_after}")

Skewness after transformation:
COMMONAREA_AVG    4.18
DAYS_EMPLOYED      NaN
OWN_CAR_AGE      -0.30
AMT_CREDIT        1.23
APARTMENTS_AVG    1.99
dtype: float64
Kurtosis for COMMONAREA_AVG after transformation: 26.47
Kurtosis for DAYS_EMPLOYED after transformation: nan
Kurtosis for OWN_CAR_AGE after transformation: 0.3
Kurtosis for AMT_CREDIT after transformation: 1.93
Kurtosis for APARTMENTS_AVG after transformation: 6.13


After transformation, the skewness and kurtosis values are reduced significantly. This makes the data more suitable for analysis.

In [87]:
# Plot the boxplots of each the numeric features for outlier analysis
plt.figure(figsize=(15, 15))
for i, feature in enumerate(numeric_features, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(x=df['TARGET'], y=df[feature], hue=df['TARGET'])
    plt.title(f"{feature} vs Target")
plt.tight_layout()
plt.show()

After transformation, outliers are reduced in the numeric features. The boxplots help identify underlying patterns in the data which couldn't be seen before transformation.

In [88]:
for feature in numeric_features:
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers_count = ((df[feature] < lower_bound) | (df[feature] > upper_bound)).sum()
    outlier_counts[feature] = outliers_count
sorted_outlier_counts = sorted(outlier_counts.items(), key=lambda x: x[1], reverse=True)

print("Number of outliers for each numeric variable (descending order):")
for feature, count in sorted_outlier_counts:
    print(f"{feature}: {count}")

Number of outliers for each numeric variable (descending order):
APARTMENTS_AVG: 8895
COMMONAREA_AVG: 7563
AMT_CREDIT: 6562
OWN_CAR_AGE: 2137
DAYS_EMPLOYED: 3


It looks like the number of outliers have reduced significantly for all these variables and the spread of the box plots have increased

## 7. Categorical Features

In [89]:
# Selecting five categorical features for analysis
categorical_features = ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS']

# Check cardinality and rare values for each categorical feature
for feature in categorical_features:
    print(f"Feature: {feature}")
    print(f"Cardinality: {df[feature].nunique()}")
    print(f"Unique values: {df[feature].unique()}")
    print("Value counts:")
    print(df[feature].value_counts(normalize=True).round(3))
    print()

# Discuss whether each feature is ordinal or nominal
ordinal_features = ['NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE']
nominal_features = ['NAME_CONTRACT_TYPE', 'CODE_GENDER','NAME_FAMILY_STATUS' ]

Feature: NAME_CONTRACT_TYPE
Cardinality: 2
Unique values: ['Cash loans' 'Revolving loans']
Value counts:
NAME_CONTRACT_TYPE
Cash loans         0.905
Revolving loans    0.095
Name: proportion, dtype: float64

Feature: CODE_GENDER
Cardinality: 3
Unique values: ['M' 'F' 'XNA']
Value counts:
CODE_GENDER
F      0.658
M      0.342
XNA    0.000
Name: proportion, dtype: float64

Feature: NAME_INCOME_TYPE
Cardinality: 8
Unique values: ['Working' 'State servant' 'Commercial associate' 'Pensioner' 'Unemployed'
 'Student' 'Businessman' 'Maternity leave']
Value counts:
NAME_INCOME_TYPE
Working                 0.516
Commercial associate    0.233
Pensioner               0.180
State servant           0.071
Unemployed              0.000
Student                 0.000
Businessman             0.000
Maternity leave         0.000
Name: proportion, dtype: float64

Feature: NAME_EDUCATION_TYPE
Cardinality: 5
Unique values: ['Secondary / secondary special' 'Higher education' 'Incomplete higher'
 'Lower seconda

Categorical features were randomly chosen from the data. Contract Type, Gender, Family Status type are all nominal variables whereas Education Type and Income Type are ordinal features. Cardinality and unique counts are displayed above

## 8. Feature Engineering

In [90]:
# Feature 1: Count of previous applications per SK_ID_CURR
df_previous = pd.read_csv("/content/drive/MyDrive/home-credit-default-risk/previous_application.csv",header=0)
previous_count = df_previous.groupby('SK_ID_CURR').size().reset_index(name='PREV_APP_COUNT')
df = df.merge(previous_count, on='SK_ID_CURR', how='left')
df.head(5)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,PREV_APP_COUNT
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,NaN,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.024400,0.0369,0.9722,0.6192,0.014199,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,NaN,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.091576,0.0529,0.9851,0.7960,0.058740,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,NaN,-4260.0,-2531,3.295837,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

The count of previous credit applications indicate how many times a person has applied for a credit. Higher count indicates a higher risk.

In [91]:
#Feature 2: Total Number of Active Loans for a particular SK_ID_CURR
bureau_df = pd.read_csv("/content/drive/MyDrive/home-credit-default-risk/bureau.csv",header=0)
new_feature_1 = bureau_df[bureau_df['CREDIT_ACTIVE'] == 'Active'].groupby('SK_ID_CURR').size()
new_feature_1.name = 'ACTIVE_LOAN_COUNT'
df = df.merge(new_feature_1, on='SK_ID_CURR', how='left')
df.head(5)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,PREV_APP_COUNT,ACTIVE_LOAN_COUNT
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,NaN,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.024400,0.0369,0.9722,0.6192,0.014199,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,NaN,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.091576,0.0529,0.9851,0.7960,0.058740,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,NaN,-4260.0,-2531,3.295837,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

Similar to the previous credit applications, the active loan count is an indicator of the current active loan applications. Higher the count, higher the risk.

In [92]:
# Feature 3: Average Number of Days Past Due per SK_ID_CURR
POS_balance_df = pd.read_csv("/content/drive/MyDrive/home-credit-default-risk/POS_CASH_balance.csv",header=0)
new_feature_2 = POS_balance_df.groupby('SK_ID_CURR')['SK_DPD'].median()
new_feature_2.name = 'AVG_DAYS_PAST_DUE'
df = df.merge(new_feature_2, on='SK_ID_CURR', how='left')
df.head(5)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,PREV_APP_COUNT,ACTIVE_LOAN_COUNT,AVG_DAYS_PAST_DUE
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,NaN,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.024400,0.0369,0.9722,0.6192,0.014199,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,NaN,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.091576,0.0529,0.9851,0.7960,0.058740,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,NaN,-4260.0,-2531,3.295837,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

 This feature give the average number of days a person has been late on their payments. Higher the number of days indicates they often pay late, which could reflect a higher risk. We select the median here as mean might be sensitive to outliers.

In [93]:
# Feature 4: Average Number of Remaining Installments per SK_ID_CURR
new_feature_3 = POS_balance_df.groupby('SK_ID_CURR')['CNT_INSTALMENT_FUTURE'].median().round(2)
new_feature_3.name = 'AVG_REMAINING_INSTALLMENTS'
df = df.merge(new_feature_3, on='SK_ID_CURR', how='left')
df.head(5)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,PREV_APP_COUNT,ACTIVE_LOAN_COUNT,AVG_DAYS_PAST_DUE,AVG_REMAINING_INSTALLMENTS
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,NaN,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.024400,0.0369,0.9722,0.6192,0.014199,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,15.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,NaN,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.091576,0.0529,0.9851,0.7960,0.058740,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,6.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,NaN,-4260.0,-2531,3.295837,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,N

This feature calculates the average number of payments a person still needs to make on their loans. It helps us understand how long they will still be in debt, which can indicate their ability to handle new credit. We select the median here as mean could be skewed due to outliers.

In [94]:
# Feature 5: Average Monthly Credit Limit per SK_ID_CURR
CC_balance_df = pd.read_csv("/content/drive/MyDrive/home-credit-default-risk/credit_card_balance.csv",header=0)
new_feature_4 = CC_balance_df.groupby('SK_ID_CURR')['AMT_DRAWINGS_CURRENT'].median().round(2)
new_feature_4.name = 'AVG_MONTHLY_DRAWINGS'
df = df.merge(new_feature_4, on='SK_ID_CURR', how='left')
df.head(5)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,PREV_APP_COUNT,ACTIVE_LOAN_COUNT,AVG_DAYS_PAST_DUE,AVG_REMAINING_INSTALLMENTS,AVG_MONTHLY_DRAWINGS
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,NaN,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.024400,0.0369,0.9722,0.6192,0.014199,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,15.0,NaN
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,NaN,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.091576,0.0529,0.9851,0.7960,0.058740,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,6.0,NaN
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,NaN,-4260.0,-2531,3.295837,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Governmen

## 9. HANDLING NaNs

Calculating Null Count per feature

Finding those attributes that poorly correlate with Target

In [95]:
numeric_columns = df.select_dtypes(include=[np.number])

# Compute correlation (e.g., Pearson correlation)
correlation_matrix = numeric_columns.corr()

ELIMINATING THE VARIABLES WITH LEAST CORRELATION WITH TARGET

In [96]:
correlation_matrix['TARGET'].abs().sort_values().head(50)

FLAG_DOCUMENT_20                0.000215
FLAG_DOCUMENT_5                 0.000316
FLAG_CONT_MOBILE                0.000370
FLAG_MOBIL                      0.000534
FLAG_DOCUMENT_12                0.000756
AMT_REQ_CREDIT_BUREAU_WEEK      0.000788
AMT_REQ_CREDIT_BUREAU_HOUR      0.000930
FLAG_DOCUMENT_19                0.001358
FLAG_DOCUMENT_10                0.001414
FLAG_DOCUMENT_7                 0.001520
NONLIVINGAPARTMENTS_MODE        0.001557
FLAG_EMAIL                      0.001758
AMT_REQ_CREDIT_BUREAU_QRT       0.002022
SK_ID_CURR                      0.002108
FLAG_DOCUMENT_4                 0.002672
AMT_REQ_CREDIT_BUREAU_DAY       0.002704
NONLIVINGAPARTMENTS_MEDI        0.002757
LIVE_REGION_NOT_WORK_REGION     0.002819
NONLIVINGAPARTMENTS_AVG         0.003176
FLAG_DOCUMENT_17                0.003378
FLAG_DOCUMENT_21                0.003709
AMT_INCOME_TOTAL                0.003982
FLAG_DOCUMENT_11                0.004229
FLAG_DOCUMENT_9                 0.004352
FLAG_DOCUMENT_2 

In [97]:
df.drop(columns=['FLAG_DOCUMENT_20',
'FLAG_DOCUMENT_5',
'FLAG_CONT_MOBILE',
'FLAG_MOBIL',
'FLAG_DOCUMENT_12',
'AMT_REQ_CREDIT_BUREAU_WEEK',
'AMT_REQ_CREDIT_BUREAU_HOUR',
'FLAG_DOCUMENT_19',
'FLAG_DOCUMENT_10',
'FLAG_DOCUMENT_7',
'NONLIVINGAPARTMENTS_MODE',
'FLAG_EMAIL',
'AMT_REQ_CREDIT_BUREAU_QRT',
'SK_ID_CURR',
'FLAG_DOCUMENT_4',
'AMT_REQ_CREDIT_BUREAU_DAY',
'NONLIVINGAPARTMENTS_MEDI',
'LIVE_REGION_NOT_WORK_REGION',
'NONLIVINGAPARTMENTS_AVG',
'FLAG_DOCUMENT_17',
'FLAG_DOCUMENT_21',
'AMT_INCOME_TOTAL',
'FLAG_DOCUMENT_11',
'FLAG_DOCUMENT_9',
'FLAG_DOCUMENT_2',
'AVG_DAYS_PAST_DUE',
'REG_REGION_NOT_LIVE_REGION',
'FLAG_DOCUMENT_15',
'REG_REGION_NOT_WORK_REGION',
'FLAG_DOCUMENT_18',
'FLAG_DOCUMENT_8',
'OBS_60_CNT_SOCIAL_CIRCLE',
'YEARS_BEGINEXPLUATATION_MODE',
'OBS_30_CNT_SOCIAL_CIRCLE',
'CNT_FAM_MEMBERS',
'FLAG_DOCUMENT_14',
'YEARS_BEGINEXPLUATATION_AVG',
'YEARS_BEGINEXPLUATATION_MEDI',
'LANDAREA_MODE',
'LANDAREA_AVG',
'LANDAREA_MEDI',
'FLAG_DOCUMENT_13',
'FLAG_DOCUMENT_16',
'DAYS_EMPLOYED',
'AMT_REQ_CREDIT_BUREAU_MON',
'NONLIVINGAREA_MODE',
'AMT_ANNUITY',
'NONLIVINGAREA_MEDI',
'NONLIVINGAREA_AVG',
'COMMONAREA_MODE','FLAG_DOCUMENT_3',
'FLAG_DOCUMENT_6'], axis=1, inplace=True)

CONVERTING VARIABLES INTO APPROPRIATE DATATYPES (CATEGORICAL)

In [98]:
df[['TARGET','NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_TYPE_SUITE','NAME_INCOME_TYPE',
       'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FLAG_EMP_PHONE',
       'FLAG_WORK_PHONE', 'FLAG_PHONE', 'OCCUPATION_TYPE',
       'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY',
       'WEEKDAY_APPR_PROCESS_START',
       'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY',
       'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE','WALLSMATERIAL_MODE','FONDKAPREMONT_MODE', 'HOUSETYPE_MODE','EMERGENCYSTATE_MODE','DEF_30_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE']]=df[['TARGET','NAME_CONTRACT_TYPE',
       'CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_TYPE_SUITE','NAME_INCOME_TYPE',
       'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FLAG_EMP_PHONE',
       'FLAG_WORK_PHONE', 'FLAG_PHONE', 'OCCUPATION_TYPE',
       'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY',
       'WEEKDAY_APPR_PROCESS_START',
       'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY',
       'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE','WALLSMATERIAL_MODE','FONDKAPREMONT_MODE', 'HOUSETYPE_MODE','EMERGENCYSTATE_MODE','DEF_30_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE']].astype('category')

In [99]:
num_cols= ['COMMONAREA_AVG',
        'LIVINGAPARTMENTS_MEDI',
        'COMMONAREA_MEDI',
        'LIVINGAPARTMENTS_MODE',
        'LIVINGAPARTMENTS_AVG',
        'FLOORSMIN_MODE',
        'FLOORSMIN_MEDI',
        'FLOORSMIN_AVG',
        'EXT_SOURCE_1',
        'EXT_SOURCE_3',
        'AMT_REQ_CREDIT_BUREAU_YEAR',
        'YEARS_BUILD_AVG',
        'YEARS_BUILD_MODE',
        'YEARS_BUILD_MEDI',
        'OWN_CAR_AGE',
        'BASEMENTAREA_AVG',
        'BASEMENTAREA_MODE',
        'BASEMENTAREA_MEDI',
        'ELEVATORS_MODE',
        'ELEVATORS_MEDI',
        'ELEVATORS_AVG',
        'APARTMENTS_AVG',
        'APARTMENTS_MEDI',
        'APARTMENTS_MODE',
        'ENTRANCES_MODE',
        'ENTRANCES_MEDI',
        'ENTRANCES_AVG',
        'LIVINGAREA_MEDI',
        'LIVINGAREA_MODE',
        'LIVINGAREA_AVG',
        'FLOORSMAX_AVG',
        'FLOORSMAX_MEDI',
        'FLOORSMAX_MODE',
        'TOTALAREA_MODE',
        'DAYS_LAST_PHONE_CHANGE','AMT_GOODS_PRICE', 'EXT_SOURCE_2']
for col in num_cols:
  col_mean= df[col].mean()
  df[col].fillna(col_mean,inplace=True)

df.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_CREDIT,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,OCCUPATION_TYPE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BUILD_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_YEAR,PREV_APP_COUNT,ACTIVE_LOAN_COUNT,AVG_REMAINING_INSTALLMENTS,AVG_MONTHLY_DRAWINGS
0,1,Cash loans,M,N,Y,0,406597.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-3648.0,-2120,2.246389,1,0,1,Laborers,2,2,WEDNESDAY,10,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.024400,0.036900,0.619200,0.014199,0.000000,0.069000,0.083300,0.125000,0.020200,0.019000,0.025200,0.038300,0.634100,0.00000,0.069000,0.083300,0.125000,0.022000,0.019800,0.02500,0.036900,0.624300,0.014400,0.000000,0.069000,0.083300,0.125000,0.020500,0.019300,reg oper account,block of flats,0.014900,"Stone, brick",No,2.0,2.0,-1134.0,1.000000,1.0,2.0,15.0,NaN
1,0,Cash loans,F,N,N,0,1293502.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1186.0,-291,2.246389,1,0,1,Core staff,1,1,MONDAY,11,0,0,0,School,0.311267,0.622246,0.510853,0.091576,0.052900,0.796000,0.058740,0.080000,0.034500,0.291700,0.333300,0.077300,0.054900,0.092400,0.053800,0.804000,0.08060,0.034500,0.291700,0.333300,0.079000,0.055400,0.09680,0.052900,0.798700,0.060800,0.080000,0.034500,0.291700,0.333300,0.078700,0.055800,reg oper account,block of flats,0.071400,Block,No,0.0,0.0,-828.0,0.000000,3.0,1.0,6.0,NaN
2,0,Revolving loans,M,Y,Y,0,135000.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-4260.0,-2531,3.295837,1,1,1,Laborers,2,2,MONDAY,9,0,0,0,Government,0.502130,0.555912,0.729567,0.106929,0.088442,0.752471,0.041488,0.078942,0.149725,0.226282,0.231894,0.100775,0.107399,0.114231,0.087543,0.759637,0.07449,0.145193,0.222315,0.228058,0.105645,0.105975,0.11785,0.087955,0.755746,0.044595,0.078078,0.149213,0.225897,0.231625,0.101954,0.108607,NaN,NaN,0.102547,NaN,NaN,0.0,0.0,-815.0,0.000000,1.0,NaN,2.5,NaN
3,0,Cash loans,F,N,Y,0,312682.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-9833.0,-2437,2.246389,1,0,0,Laborers,2,2,WEDNESDAY,17,0,0,0,Business Entity Type 3,0.502130,0.650442,0.510853,0.106929,0.088442,0.752471,0.041488,0.078942,0.149725,0.226282,0.231894,0.100775,0.107399,0.114231,0.087543,0.759637,0.07449,0.145193,0.222315,0.228058,0.105645,0.105975,0.11785,0.087955,0.755746,0.044595,0.078078,0.149213,0.225897,0.231625,0.101954,0.108607,NaN,NaN,0.102547,NaN,NaN,0.0,0.0,-617.0,1.899974,9.0,NaN,8.0,0.0
4,0,Cash loans,M,N,Y,0,513000.0,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-4311.0,-3458,2.246389,1,0,0,Core staff,2,2,THURSDAY,11,0,1,1,Religion,0.502130,0.322738,0.510853,0.106929,0.088442,0.752471,0.041488,0.078942,0.149725,0.226282,0.231894,0.100775,0.107399,0.114231,0.087543,0.759637,

In [100]:
selected_columns = ['PREV_APP_COUNT', 'ACTIVE_LOAN_COUNT', 'AVG_REMAINING_INSTALLMENTS','AVG_MONTHLY_DRAWINGS']

# Fill null values with 0 in the selected columns
df[selected_columns] = df[selected_columns].fillna(0)

In [101]:
df[['TARGET',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'OCCUPATION_TYPE',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'DEF_60_CNT_SOCIAL_CIRCLE']].isna().sum()

TARGET                              0
NAME_CONTRACT_TYPE                  0
CODE_GENDER                         0
FLAG_OWN_CAR                        0
FLAG_OWN_REALTY                     0
NAME_TYPE_SUITE                  1292
NAME_INCOME_TYPE                    0
NAME_EDUCATION_TYPE                 0
NAME_FAMILY_STATUS                  0
NAME_HOUSING_TYPE                   0
FLAG_EMP_PHONE                      0
FLAG_WORK_PHONE                     0
FLAG_PHONE                          0
OCCUPATION_TYPE                 96391
REGION_RATING_CLIENT                0
REGION_RATING_CLIENT_W_CITY         0
WEEKDAY_APPR_PROCESS_START          0
REG_CITY_NOT_LIVE_CITY              0
REG_CITY_NOT_WORK_CITY              0
LIVE_CITY_NOT_WORK_CITY             0
ORGANIZATION_TYPE                   0
FONDKAPREMONT_MODE             210295
HOUSETYPE_MODE                 154297
WALLSMATERIAL_MODE             156341
EMERGENCYSTATE_MODE            145755
DEF_30_CNT_SOCIAL_CIRCLE         1021
DEF_60_CNT_S

In [102]:
cat_col = df.select_dtypes(include='category').columns
for col in cat_col:
  mode_col= df[col].mode()[0]
  df[col].fillna(mode_col,inplace=True)

In [103]:
df.columns[df.isnull().any()].tolist()

[]

### 1. Which type of machine learning problem is it?

The primary objective is to predict whether a borrower will default on a loan. This type of prediction task falls into that of binary classification. The goal is to classify borrowers into two categories: those likely to default and those not likely to default.

### 2. Suggest at least three models and discuss their pros and cons

a) Logistic Regression:

Pros:
Simple and interpretable model.
Efficient to train and can handle large datasets.
Outputs probabilities which can be used for risk assessment.

Cons:
Assumes linear relationship between features and log-odds of the target.
May underperform if the decision boundary is highly non-linear.

b) Random Forest Classifier:

Pros:
Robust and less prone to overfitting compared to decision trees.
Handles non-linear relationships well.
Automatically handles feature selection and interaction.

Cons:
Can be computationally expensive and slow to train, especially with large datasets.
Less interpretable compared to simpler models like logistic regression.

c) Decision Tree:

Pros:
Handles complex interactions between features.
Can handle missing data and outliers well.

Cons:
Model can exhibit high variance in terms of replicating results with another set of parameters.

There are almost 75 variables in the dataset. We use Autoviz to list the top 30 variables using XG Boost and find the outliers/ rare categories in each

In [ ]:
AV = AutoViz_Class()
AV.AutoViz("", depVar= "TARGET",dfte = df)

    Since nrows is smaller than dataset, loading random sample of 150000 rows into pandas...
Shape of your Data Set loaded: (150000, 75)
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
Classifying variables in data set...
    Number of Numeric Columns =  44
    Number of Integer-Categorical Columns =  4
    Number of String-Categorical Columns =  0
    Number of Factor-Categorical Columns =  26
    Number of String-Boolean Columns =  0
    Number of Numeric-Boolean Columns =  0
    Number of Discrete String Columns =  0
    Number of NLP String Columns =  0
    Number of Date Time Columns =  0
    Number of ID Columns =  0
    Number of Columns to Delete =  0
    74 Predictors classified...
        No variables removed since no ID or low-information variables found in data 

,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
REGION_POPULATION_RELATIVE,float64,0.000000,NA,0.000290,0.072508,Column has 4188 outliers greater than upper bound (0.06) or lower than lower bound(-0.02). Cap them or remove them.
DAYS_REGISTRATION,float64,0.000000,NA,-24672.000000,0.000000,Column has 306 outliers greater than upper bound (6179.00) or lower than lower bound(-15709.00). Cap them or remove them.
OWN_CAR_AGE,float64,0.000000,NA,0.000000,91.000000,Column has 51110 outliers greater than upper bound (12.06) or lower than lower bound(12.06). Cap them or remove them.
EXT_SOURCE_1,float64,0.000000,NA,0.014568,0.962693,Column has 65573 outliers greater than upper bound (0.50) or lower than lower bound(0.50). Cap them or remove them.
EXT_SOURCE_2,float64,0.000000,NA,0.000005,0.855000,No issue
EXT_SOURCE_3,float64,0.000000,NA,0.000527,0.896010,Column has 2154 outliers greater than upper bound (0.96) or lower than lower bound(0.09). Cap them or remove them.
DAYS_LAST_PHONE_CHANGE,float64,0.000000,NA,-4292.000000,0.000000,Column has 218 outliers greater than upper bound (1663.50) or lower than lower bound(-3508.50). Cap them or remove them.
PREV_APP_COUNT,float64,0.000000,NA,0.000000,73.000000,Column has 7459 outliers greater than upper bound (12.00) or lower than lower bound(-4.00). Cap them or remove them.
ACTIVE_LOAN_COUNT,float64,0.000000,NA,0.000000,19.000000,Column has 1609 outliers greater than upper bound (7.50) or lower than lower bound(-4.50). Cap them or remove them.
AVG_REMAINING_INSTALLMENTS,float64,0.000000,NA,0.000000,60.000000,Column has 12191 outliers greater than upper bound (17.75) or lower than lower bound(-4.25). Cap them or remove them.


KeyboardInterrupt: 

Error in callback <function _draw_all_if_interactive at 0x788b5704c5e0> (for post_execute):


KeyboardInterrupt: 

These are the variables listed by Autoviz and we classify them into different types of variables

In [104]:
num_vars = ['REGION_POPULATION_RELATIVE','DAYS_REGISTRATION','OWN_CAR_AGE','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_LAST_PHONE_CHANGE',
            'PREV_APP_COUNT','ACTIVE_LOAN_COUNT','AVG_REMAINING_INSTALLMENTS','AVG_MONTHLY_DRAWINGS','AMT_REQ_CREDIT_BUREAU_YEAR','COMMONAREA_MEDI',
            'FLOORSMIN_AVG','YEARS_BUILD_MODE','FLOORSMAX_MEDI','ENTRANCES_MODE','LIVINGAPARTMENTS_AVG','BASEMENTAREA_MODE','LIVINGAREA_MODE','AMT_CREDIT','CNT_CHILDREN']
# Nominal categorical variables from the dataset
nom_cat_vars = ['NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_FAMILY_STATUS','NAME_INCOME_TYPE','NAME_TYPE_SUITE'] # None

# Ordinal categorical variables from the dataset
ord_cat_vars = ['NAME_EDUCATION_TYPE'] # There is none thus kept blank. If for a new dataset, you find some relevant variables pass them here

# Categorical variables with rare categories
rare_cat_vars = ['CODE_GENDER','NAME_TYPE_SUITE','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS']

# Numeric variables that require discretization
disc_num_vars = [] # None here

# Numeric variables that are normally distributed
norm_num_vars = [] # None here

# Numeric variables that ahave skew in them



We build a pipeline to handle the outliers for each of these variable types and then fit them on the newly created dataframe with the 30 variables , excluding TARGET

In [105]:
num_pipe  = Pipeline(steps = [("out", Winsorizer(capping_method='quantiles', tail='both', fold=.06))])
# Nominal Categorical Variables
nom_cat_pipe = Pipeline(steps = [("ohe", OneHotEncoder(sparse_output=False, drop='first',handle_unknown='ignore', categories='auto'))])
# Ordinal Catgeorical variables
ord_cat_pipe = Pipeline(steps = [("ord", OrdinalEncoder(categories='auto',handle_unknown='error'))])
rare_cat_pipe = Pipeline(steps = [("rare", RareLabelEncoder(tol=0.05, n_categories=5)), ("ohe", OneHotEncoder(sparse_output=False,handle_unknown='ignore'))])

In [106]:
preprocessor = ColumnTransformer(transformers = [("nom", nom_cat_pipe, nom_cat_vars),
                                                 ("ord", ord_cat_pipe, ord_cat_vars),
                                                 ("rare", rare_cat_pipe, rare_cat_vars),
                                                 ("skew", num_pipe,num_vars)], remainder = "passthrough")


preprocessor.set_output(transform = "pandas")

ColumnTransformer(remainder='passthrough',
                  transformers=[('nom',
                                 Pipeline(steps=[('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['NAME_CONTRACT_TYPE', 'CODE_GENDER',
                                  'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
                                  'NAME_FAMILY_STATUS', 'NAME_INCOME_TYPE',
                                  'NAME_TYPE_SUITE']),
                                ('ord',
                                 Pipeline(steps=[('ord', OrdinalEncoder())]),
                                 ['NAME_EDU...
                                  'EXT_SOURCE_1', 'EXT_SOURCE_2',
                                  'EXT_SOURCE_3', 'DAYS_LAST_PHONE_CHANGE',
                                  'PREV_APP_COUNT', 'ACTIVE_LOAN_COUNT',
                                  'AVG_REMAINING_INSTALLMENTS',
                                  'AVG_MONTHLY_DRAWINGS',
                                  'AMT_REQ_CREDIT_BUREAU_YEAR',
                                  'COMMONAREA_MEDI', 'FLOORSMIN_AVG',
                                  'YEARS_BUILD_MODE', 'FLOORSMAX_MEDI',
                                  'ENTRANCES_MODE', 'LIVINGAPARTMENTS_AVG',
                                  'BASEMENTAREA_MODE', 'LIVINGAREA_MODE',
                                  'AMT_CREDIT', 'CNT_CHILDREN'])])

In [107]:
model_df= df[['REGION_POPULATION_RELATIVE','DAYS_REGISTRATION','OWN_CAR_AGE','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_LAST_PHONE_CHANGE',
            'PREV_APP_COUNT','ACTIVE_LOAN_COUNT','AVG_REMAINING_INSTALLMENTS','AVG_MONTHLY_DRAWINGS','AMT_REQ_CREDIT_BUREAU_YEAR','COMMONAREA_MEDI',
            'FLOORSMIN_AVG','YEARS_BUILD_MODE','FLOORSMAX_MEDI','ENTRANCES_MODE','LIVINGAPARTMENTS_AVG','BASEMENTAREA_MODE','LIVINGAREA_MODE','AMT_CREDIT','CNT_CHILDREN',
             'NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_FAMILY_STATUS','NAME_INCOME_TYPE','NAME_TYPE_SUITE',
              'NAME_EDUCATION_TYPE', 'TARGET']]

In [108]:
preprocessor.fit(model_df)
df2 = preprocessor.transform(model_df)

In [109]:
df2['remainder__TARGET']= df2['remainder__TARGET'].astype('int64')

In [110]:
X = df2.drop(columns=['remainder__TARGET'])  # Features
y = df2['remainder__TARGET']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Grid search parameters for finding the best model settings

In [111]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# Define parameter grid for Random Forest Classifier
rf_param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [4, 8],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [1, 2],
    'n_estimators': [100]
}

# Define parameter grid for Decision Tree Classifier
dt_param_grid = {
    'max_depth': [5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Define parameter grid for Logistic Regression
logreg_param_grid = {
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'saga']
}

Creating a logistic, decision tree and random forest model variable

In [112]:
rf_clf = RandomForestClassifier(class_weight='balanced', random_state=42)
dt_clf = DecisionTreeClassifier()
logreg_clf = LogisticRegression()

Conducting a grid search on each of these models to estimate the optimal parameter combination to get the maximum accuracy model

In [113]:
logreg_grid_search = GridSearchCV(logreg_clf, logreg_param_grid, scoring='accuracy', cv=5)
logreg_grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.1, 1, 10], 'solver': ['liblinear', 'saga']},
             scoring='accuracy')

In [114]:
dt_grid_search = GridSearchCV(dt_clf, dt_param_grid, scoring='accuracy', cv=5)
dt_grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [5, 10], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5]},
             scoring='accuracy')

In [115]:
rf_grid_search = GridSearchCV(rf_clf, rf_param_grid, scoring='accuracy', cv=5)
rf_grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=42),
             param_grid={'criterion': ['gini', 'entropy'], 'max_depth': [4, 8],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [5, 10], 'n_estimators': [100]},
             scoring='accuracy')

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [116]:
# Best performing Logistic Regression
print("Best Logistic Regression:")
print("Best Parameters:", logreg_grid_search.best_params_)
print("Best CV Score (Accuracy):", logreg_grid_search.best_score_)

Best Logistic Regression:
Best Parameters: {'C': 0.1, 'solver': 'liblinear'}
Best CV Score (Accuracy): 0.9192058795086311


In [117]:
print("Best Decision Tree Classifier:")
print("Best Parameters:", dt_grid_search.best_params_)
print("Best CV Score (Accuracy):", dt_grid_search.best_score_)

Best Decision Tree Classifier:
Best Parameters: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best CV Score (Accuracy): 0.9191205171249882


In [118]:
# Best performing Random Forest Classifier
print("Best Random Forest Classifier:")
print("Best Parameters:", rf_grid_search.best_params_)
print("Best CV Score (Accuracy):", rf_grid_search.best_score_)

Best Random Forest Classifier:
Best Parameters: {'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Best CV Score (Accuracy): 0.6922701822396395


The prediction model with the highest accuracy is the logistic regression model with an accuracy of 91.92%.

In [119]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report
# Evaluating the logistic regression model on the training data
y_train_pred_lr = logreg_grid_search.predict(X_train)
train_accuracy_lr = accuracy_score(y_train, y_train_pred_rf)
print(" Logistic Regression Training Accuracy:", train_accuracy_lr)
print("Confusion Matrix on Train Data:\n", confusion_matrix(y_train, y_train_pred_lr))

# Evaluating the logistic model on the test data
y_test_pred_lr = logreg_grid_search.predict(X_test)
test_accuracy_lr = accuracy_score(y_test, y_test_pred_lr)
print("Logistic Regression Test Accuracy:", test_accuracy_lr)
print("Confusion Matrix on Test Data:\n", confusion_matrix(y_test, y_test_pred_lr))

 Logistic Regression Training Accuracy: 0.919205879483594
Confusion Matrix on Train Data:
 [[226132      0]
 [ 19876      0]]
Logistic Regression Test Accuracy: 0.9195323805342829
Confusion Matrix on Test Data:
 [[56554     0]
 [ 4949     0]]


In [120]:
# Evaluating the decision tree regression model on the training data
y_train_pred_dt = dt_grid_search.predict(X_train)
train_accuracy_dt = accuracy_score(y_train, y_train_pred_dt)
print(" Decision Tree Training Accuracy:", train_accuracy_dt)
print("Confusion Matrix on Train Data:\n", confusion_matrix(y_train, y_train_pred_dt))

# Evaluating the decision tree model on the test data
y_test_pred_dt = dt_grid_search.predict(X_test)
test_accuracy_dt = accuracy_score(y_test, y_test_pred_dt)
print("Decision Tree Test Accuracy:", test_accuracy_dt)
print("Confusion Matrix on Test Data:\n", confusion_matrix(y_test, y_test_pred_dt))

 Decision Tree Training Accuracy: 0.919205879483594
Confusion Matrix on Train Data:
 [[226132      0]
 [ 19876      0]]
Decision Tree Test Accuracy: 0.9195323805342829
Confusion Matrix on Test Data:
 [[56554     0]
 [ 4949     0]]


In [121]:
# Evaluating the random forest model on the training data
y_train_pred_rf = dt_grid_search.predict(X_train)
train_accuracy_rf = accuracy_score(y_train, y_train_pred_rf)
print(" Decision Tree Training Accuracy:", train_accuracy_rf)
print("Confusion Matrix on Train Data:\n", confusion_matrix(y_train, y_train_pred_rf))

# Evaluating the random forest model on the test data
y_test_pred_rf = rf_grid_search.predict(X_test)
test_accuracy_rf = accuracy_score(y_test, y_test_pred_rf)
print("Decision Tree Test Accuracy:", test_accuracy_rf)
print("Confusion Matrix on Test Data:\n", confusion_matrix(y_test, y_test_pred_rf))

 Decision Tree Training Accuracy: 0.919205879483594
Confusion Matrix on Train Data:
 [[226132      0]
 [ 19876      0]]
Decision Tree Test Accuracy: 0.6808936149456124
Confusion Matrix on Test Data:
 [[38585 17969]
 [ 1657  3292]]


In [ ]:
def class_rep(y_true, y_pred):
    class_report = classification_report(y_true, y_pred)
    print("Classification Report:")
    print(class_report)
print('Logistic Regression Classification Report')
class_rep(y_test, y_test_pred_lr)

print('Decision Tree Classification Report')
class_rep(y_test, y_test_pred_dt)

print('Random Forest Classification Report')
class_rep(y_test, y_test_pred_rf)

Logistic Regression Classification Report
Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56554
           1       0.00      0.00      0.00      4949

    accuracy                           0.92     61503
   macro avg       0.46      0.50      0.48     61503
weighted avg       0.85      0.92      0.88     61503

Decision Tree Classification Report
Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56554
           1       0.00      0.00      0.00      4949

    accuracy                           0.92     61503
   macro avg       0.46      0.50      0.48     61503
weighted avg       0.85      0.92      0.88     61503

Random Forest Classification Report
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.68      0.80     56554
           1       0.15      0.67      0.25      4949


In [126]:
from sklearn.metrics import roc_auc_score

probabilities = rf_grid_search.predict_proba(X_test)[:, 1]  # get probabilities for the positive class

# Compute ROC-AUC
roc_auc = roc_auc_score(y_test, probabilities)
print(f'ROC-AUC Score: {roc_auc}')

probabilities_lr = logreg_grid_search.predict_proba(X_test)[:, 1]  # get probabilities for the positive class

# Compute ROC-AUC
roc_auc_lr = roc_auc_score(y_test, probabilities_lr)
print(f'ROC-AUC Score: {roc_auc_lr}')


probabilities_dt = dt_grid_search.predict_proba(X_test)[:, 1]
roc_auc_dt = roc_auc_score(y_test, probabilities_dt)
print(f'ROC-AUC Score: {roc_auc_dt}')


ROC-AUC Score: 0.735584231931554
ROC-AUC Score: 0.577091135609314
ROC-AUC Score: 0.7108115555838275


The ROC-AUC score, standing for "Receiver Operating Characteristic - Area Under the Curve," is a performance measurement for classification problems at various threshold settings. It's particularly useful for evaluating binary classification models and provides insights into how well a model can distinguish between two classes.



If a random positive (default) and  negative (non-default) person is picked then, there is a 73.55% / 57.7% OR 71% chance that the model will be able to correctly distinguish the positive from the negative one.

In [ ]:
submissions_df = pd.DataFrame(X_test)
submissions_df['Actual_TARGET_Test'] = y_test
submissions_df['TARGET_pred_lr'] = y_test_pred_lr
submissions_df['TARGET_pred_dt'] = y_test_pred_dt
submissions_df['TARGET_pred_rf'] = y_test_pred_rf
submissions_df.head(5)

,nom__NAME_CONTRACT_TYPE_Revolving loans,nom__CODE_GENDER_M,nom__CODE_GENDER_XNA,nom__FLAG_OWN_CAR_Y,nom__FLAG_OWN_REALTY_Y,nom__NAME_FAMILY_STATUS_Married,nom__NAME_FAMILY_STATUS_Separated,nom__NAME_FAMILY_STATUS_Single / not married,nom__NAME_FAMILY_STATUS_Unknown,nom__NAME_FAMILY_STATUS_Widow,nom__NAME_INCOME_TYPE_Commercial associate,nom__NAME_INCOME_TYPE_Maternity leave,nom__NAME_INCOME_TYPE_Pensioner,nom__NAME_INCOME_TYPE_State servant,nom__NAME_INCOME_TYPE_Student,nom__NAME_INCOME_TYPE_Unemployed,nom__NAME_INCOME_TYPE_Working,nom__NAME_TYPE_SUITE_Family,nom__NAME_TYPE_SUITE_Group of people,nom__NAME_TYPE_SUITE_Other_A,nom__NAME_TYPE_SUITE_Other_B,"nom__NAME_TYPE_SUITE_Spouse, partner",nom__NAME_TYPE_SUITE_Unaccompanied,ord__NAME_EDUCATION_TYPE,rare__CODE_GENDER_F,rare__CODE_GENDER_M,rare__CODE_GENDER_XNA,rare__NAME_TYPE_SUITE_Family,rare__NAME_TYPE_SUITE_Rare,rare__NAME_TYPE_SUITE_Unaccompanied,rare__NAME_INCOME_TYPE_Commercial associate,rare__NAME_INCOME_TYPE_Pensioner,rare__NAME_INCOME_TYPE_Rare,rare__NAME_INCOME_TYPE_State servant,rare__NAME_INCOME_TYPE_Working,rare__NAME_EDUCATION_TYPE_Academic degree,rare__NAME_EDUCATION_TYPE_Higher education,rare__NAME_EDUCATION_TYPE_Incomplete higher,rare__NAME_EDUCATION_TYPE_Lower secondary,rare__NAME_EDUCATION_TYPE_Secondary / secondary special,rare__NAME_FAMILY_STATUS_Civil marriage,rare__NAME_FAMILY_STATUS_Married,rare__NAME_FAMILY_STATUS_Rare,rare__NAME_FAMILY_STATUS_Separated,rare__NAME_FAMILY_STATUS_Single / not married,rare__NAME_FAMILY_STATUS_Widow,skew__REGION_POPULATION_RELATIVE,skew__DAYS_REGISTRATION,skew__OWN_CAR_AGE,skew__EXT_SOURCE_1,skew__EXT_SOURCE_2,skew__EXT_SOURCE_3,skew__DAYS_LAST_PHONE_CHANGE,skew__PREV_APP_COUNT,skew__ACTIVE_LOAN_COUNT,skew__AVG_REMAINING_INSTALLMENTS,skew__AVG_MONTHLY_DRAWINGS,skew__AMT_REQ_CREDIT_BUREAU_YEAR,skew__COMMONAREA_MEDI,skew__FLOORSMIN_AVG,skew__YEARS_BUILD_MODE,skew__FLOORSMAX_MEDI,skew__ENTRANCES_MODE,skew__LIVINGAPARTMENTS_AVG,skew__BASEMENTAREA_MODE,skew__LIVINGAREA_MODE,skew__AMT_CREDIT,skew__CNT_CHILDREN,Actual_TARGET_Test,TARGET_pred_lr,TARGET_pred_dt,TARGET_pred_rf
245895,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.009630,-637.0,2.944439,0.675878,0.604894,0.192942,-2.0,5.0,3.0,6.0,0.0,1.0,0.044595,0.231894,0.759637,0.225897,0.145193,0.100775,0.087543,0.105975,465457.5,2,0,0,0,1
98194,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.006852,-1610.0,2.484907,0.430827,0.425351,0.712155,-1071.0,12.0,2.0,3.5,0.0,3.0,0.062600,0.291700,0.843200,0.375000,0.034500,0.060100,0.056600,0.085700,1281712.5,0,0,0,0,0
36463,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.035792,-2507.0,1.609438,0.527239,0.531760,0.207964,-1435.0,7.0,3.0,5.0,0.0,3.0,0.044595,0.231894,0.759637,0.225897,0.145193,0.100775,0.087543,0.105975,495000.0,0,0,0,0,1
249923,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.046220,-11016.0,2.246389,0.502130,0.693521,0.614414,-2000.0,1.0,0.0,0.0,0.0,0.0,0.044595,0.231894,0.759637,0.375000,0.069000,0.100775,0.067000,0.169600,254700.0,0,0,0,0,0
158389,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.018850,-7299.0,2.246389,0.654882,0.560690,0.636376,-173.0,9.0,1.0,4.5,0.0,4.0,0.044595,0.231894,0.759637,0.166700,0.172400,0.100775,0.057400,0.078000,308133.0,0,0,0,0,0


In [ ]:
submissions_df.to_csv('model_predictions.csv', index=False)

In terms of accuracy, we observe that the logistic regression model has the highest accuracy compared to the other 2 models. But we look at the ROC scores, we observe that Random Forest has the highest chance of estimating correctly. Hence, we shall be saving the random forest model as the pickle file

In [127]:
import pickle

# Save the model to disk
filename = 'finalized_model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(rf_grid_search, file)

# Confirm the model has been saved
print("Model saved successfully as", filename)

Model saved successfully as finalized_model.pkl
